In [1]:
# Bibliotecas
import requests
import os
from datetime import datetime, timedelta
import pandas as pd
import re
import shutil
from pathlib import Path

### Importando os Dados - Base de Ausências

In [ ]:
# Pasta de referência para salvamento dos arquivos
path_input = Path(r"C:\Users\Matheus\Desktop\codigo_mba")

In [ ]:


#Baixar os dados
# Função para gerar strings de mes-ano de 1018 até 0424 (formato MMYY)
def generate_month_year_range(start, end):
    start_date = datetime.strptime(start, "%m%y")
    end_date = datetime.strptime(end, "%m%y")
    date_generated = [start_date + timedelta(days=30*i) for i in range((end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1)]
    return [date.strftime("%m%y") for date in date_generated]

# Nome da pasta na qual os arquivos serão salvos
folder_name = "base"
folder_path = path_input / folder_name
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Data de início e fim do período de coleta - optei por pegar grande parte dos dados disponíveis 
start_date = "1018"
end_date = "0424"

# Gera uma lista de datas a partir das strings de data
dates = generate_month_year_range(start_date, end_date)

# Itera sobre as datas e faz requisições para o site da prefeitura da educação, extraindo os dados
for date in dates:
    url = f'https://dados.educacao.sp.gov.br/sites/default/files/%5Bdbo%5D.%5BBASE_AUSENCIAS_{date}%5D.csv'
    response = requests.get(url, verify=False)
    
    if response.status_code == 200:
        file_path = os.path.join(folder_path, f'[dbo].[BASE_AUSENCIAS_{date}].csv')
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f'Arquivo extraido para {file_path}')
    else:
        print(f'Falha na extração do arquivo {date}. Status code: {response.status_code}')

### Renomeando as Tabelas - Base de Ausências

In [9]:

#Renomear os csvs
# Diretório no qual as tabelas estão armazenadas, caso não seja a mesma onde os dados foram baixados, ajustar
directory = folder_path

# Regex para encontar o nopme bruto do arquivo
pattern = re.compile(r'\[dbo\]\.\[BASE_AUSENCIAS_(\d{4})\]')

# Itera sobre os nomes e trata eles, de forma que eles fiquem mais claros em relação ao período de referencia
for filename in os.listdir(directory):
    match = pattern.match(filename)
    if match:
        date_part = match.group(1)
        new_filename = f'ausencias_{date_part}.csv'
        old_file = os.path.join(directory, filename)
        new_file = os.path.join(directory, new_filename)
        os.rename(old_file, new_file)
        print(f'Arquivo: {old_file} renomeado para {new_file}')

### Separando os anos em pastas distintas

In [ ]:
# Separar os anos em pastas distintas
# Lista de anos para separar os dados
years = ['2019', '2020', '2021', '2022', '2023']

# Cria uma pasta para cada ano
for year in years:
    year_folder = os.path.join(directory, year)
    os.makedirs(year_folder, exist_ok=True)

# Move os arquivos para as pastas específicas, com base no ano ao qual eles se referem
for filename in os.listdir(directory):
    print(filename)
    if filename.startswith('ausencias_') and filename.endswith('.csv'):
        date_part = filename.split('_')[1].split('.')[0]
        year = f'20{date_part[2:4]}'
        if year in years:
            old_file = os.path.join(directory, filename)
            target_folder = os.path.join(directory, year)
            shutil.move(old_file, target_folder)
            print(f'Arquivo: {old_file} movido para {target_folder}')


### Combinando os CSVs com base nos anos

In [ ]:

# Agora vamos combinar os arquivos para cada ano 
#Lista de anos (igual a pasta de anos)
years = ['2019','2020', '2021', '2022', '2023']

#Itera sobre cada pasta e cria arquivos unificados de ausências, adicionando uma coluna adicional que se refere ao mês do arquivo original
for year in years:
    year_folder = os.path.join(directory,year)
    dataframes = []
    for filename in os.listdir(year_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(year_folder, filename)
            mmyy = filename.split('_')[1].split('.')[0]
            date_obj = datetime.strptime(mmyy, '%m%y')
            df = pd.read_csv(file_path,sep=';')
            df['mes'] = date_obj
            dataframes.append(df)

    # Concatena os dataframes por ano
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Salva os arquivos unificados na pasta de seus respectivos anos
    combined_df.to_excel(os.path.join(year_folder, f'{year}_combined_data.csv'), index=False)

    print(f'Arquivos concatenados salvos em {os.path.join(year_folder, "combined_data.csv")}')

### Comprimindo as bases


In [14]:
#Os arquivos originais são muito grandes, por conta de possuirem valores muito granulares (a nível de escola). Vamos somar os valores por município e por mes

folder_name = "base/bases_agregadas/bases_granulares_mes"
aggregated_data_folder = path_input / folder_name


for filename in os.listdir(aggregated_data_folder):
    year = filename.split('_')[0]
    

    file_path = os.path.join(aggregated_data_folder, filename)
    df = pd.read_csv(file_path)
    df['municipio'] = df['MUNICIPIO_EXERC'].apply(lambda x: x.strip())


    # Colunas de faltas a serem somadas
    columns_to_sum = [
        'TT_DIAS_FALTA_MEDICA', 'TT_DIAS_FALTA_JUST', 'TT_DIAS_FALTA_INJUST', 
        'TT_DIAS_LIC_PREMIO', 'TT_DIAS_LIC_GESTANTE', 'TT_DIAS_LIC_ACID_TRAB', 
        'TT_DIAS_LIC_INTER_PARTIC'
    ]



    # Os dados serão agrupados pelo nome do município e por mes, para então calcularmos o tipo de fala.
    grouped_df = df.groupby(['municipio','mes'])[columns_to_sum].sum().reset_index()

    grouped_df.to_csv(os.path.join(aggregated_data_folder, f'{year}_agg_data.csv'), index=False)

### Importando os Dados - Base de Servidores Ativos

Vamos precisar repetir o processo para extrair a quantidade de servidores ativos por município, o código é similar ao de cima

In [ ]:

#Baixar os dados
def generate_month_year_range(start, end):
    start_date = datetime.strptime(start, "%m%y")
    end_date = datetime.strptime(end, "%m%y")
    date_generated = [start_date + timedelta(days=30*i) for i in range((end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1)]
    return [date.strftime("%m%y") for date in date_generated]

# Cria uma pasta com base no primeiro nome fornecido
folder_name = "base_servidores_ativos"
folder_path = path_input / folder_name
if not os.path.exists(folder_path):
    os.makedirs(folder_path)



# Data de início e fim do período de coleta - optei por pegar grande parte dos dados disponíveis 
start_date = "1018"
end_date = "0424"

# Gera uma lista de datas a partir das strings de data
dates = generate_month_year_range(start_date, end_date)

# Itera sobre as datas e faz requisições para o site da prefeitura da educação, extraindo os dados
for date in dates:
    url = f'https://dados.educacao.sp.gov.br/sites/default/files/%5Bdbo%5D.%5BBASE_SERVIDORES_ATIVOS_{date}%5D.csv'
    response = requests.get(url, verify=False)
    
    if response.status_code == 200:
        file_path = os.path.join(folder_path, f'servidores_{date}.csv')
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f'Arquivo extraido para {file_path}')
    else:
        print(f'Falha na extração do arquivo {date}. Status code: {response.status_code}')

### Separando os anos em pastas distintas

In [ ]:
# Separar os anos em pastas distintas
# Lista de anos para separar os dados
years = ['2019', '2020', '2021', '2022', '2023']


directory =  folder_path

# pastas para cada ano
for year in years:
    year_folder = os.path.join(directory, year)
    os.makedirs(year_folder, exist_ok=True)

# Itera sobre os arquivos e os move para pastas distintas, com base em seus anos
for filename in os.listdir(directory):
    print(filename)
    if filename.startswith('servidores_') and filename.endswith('.csv'):
        date_part = filename.split('_')[1].split('.')[0]
        year = f'20{date_part[2:4]}'
        if year in years:
            old_file = os.path.join(directory, filename)
            target_folder = os.path.join(directory, year)
            shutil.move(old_file, target_folder)
            print(f'Moved: {old_file} to {target_folder}')


### Combinando os CSVs com base nos anos

De maneira similar aos dados de ausência, vamos agregar os dados por mês e municipio

In [ ]:



years = ['2019','2020', '2021', '2022', '2023']
directory =  folder_path


for year in years:
    year_folder = os.path.join(directory,year)
    dataframes = []
    for filename in os.listdir(year_folder):
        if filename.endswith('.csv'):

            file_path = os.path.join(year_folder, filename)
            print(file_path)
            mmyy = filename.split('_')[1].split('.')[0]
            
            date_obj = datetime.strptime(mmyy, '%m%y')
            
            df = pd.read_csv(file_path,sep=';')
            subset_df = df[['MUNICIPIO_E','ID_INTERNO']].copy()
                        
            subset_df_deduped =  subset_df.drop_duplicates(subset=['MUNICIPIO_E', 'ID_INTERNO']).rename(columns={
                                                                                                                    'MUNICIPIO_E': 'municipio',
                                                                                                                    'ID_INTERNO':'id_interno'
                                                                                                                    })
            dataframes.append(subset_df_deduped)


    combined_df = pd.concat(dataframes, ignore_index=True)

    combined_df_agg = combined_df['municipio'].value_counts().reset_index()
    combined_df_agg['ano'] = year
    combined_df_agg.to_excel(os.path.join(year_folder, f'{year}_combined_data.xlsx'), index=False)

    print(f'Dataframe concatenado slavo em {os.path.join(year_folder, "combined_data.csv")}')